###Disclaimer
The information contained in this notebook and any accompanying files are proprietary and is confidential to the participants of the Machine Learning Technician program and should not be copied, distributed or reproduced in whole or in part, nor passed to any third party without written permission from the Alberta Machine Intelligence Institute, Amii.

#L2-3: Data Cleaning, Alignment and Feature Engineering

## Setting up

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

## The dataset



In this lab we focus on a synthetic dataset so we can do the data cleaning steps in a reasonable amount of time.

In [2]:
#@title Run this cell to generate the synthetic datasets

%%capture
!pip install names

import names
import string

np.random.seed(100)

num_total = 100

genders = np.array(["female", "male"])
female_ones = np.random.binomial(1, 0.5, (num_total,))
male_ones = 1 - female_ones
d_gender = genders[male_ones[:, None]]
num_females = female_ones.sum()
num_males = num_total - num_females

names_list = []
for is_male in male_ones.astype('bool'):
  names_list.append(names.get_full_name(gender="male" if is_male else "female"))
d_name = np.array(names_list)

lower, upper = 0, 125
mu_female, sigma_female = 41, 20
mu_male, sigma_male = 38, 20
d_age = np.zeros((num_total, 1))
d_age[female_ones.astype('bool')] = \
  np.random.normal(mu_female, sigma_female, (num_females, 1))
d_age[male_ones.astype('bool')] = \
  np.random.normal(mu_male, sigma_male, (num_males, 1))
while np.any(np.logical_or(d_age < lower, d_age > upper)):
  d_age[d_age < lower] = 2 * lower - d_age[d_age < lower]
  d_age[d_age > upper] = 2 * upper - d_age[d_age > upper]
d_age = d_age.astype('int')
r_mu = np.array([0.056, 0, 100, 1.81, 0, 0])
s = np.array([[0.01, 1, 30, 3.16, 1, 1]]).T
c = np.array([[1, 0.28, -0.12, 0, 0.17, 0.31],
                    [0.28, 1, 0.07, 0, -0.24, 0.11],
                    [-0.12, 0.07, 1, 0, 0.04, -0.13],
                    [0, 0, 0, 1, 0, 0.42],
                    [0.17, -0.24, 0.04, 0, 1, 0.17],
                    [0.31, 0.11, -0.13, 0.42, 0.17, 1]])
r_sigma = s * c * s.T
D_b = np.random.multivariate_normal(r_mu, r_sigma, num_total)
D_b[:, 1] = np.ceil(D_b[:, 1])
D_b[:, 1] -= D_b[:, 1].min()
D_b[:, 4] = (D_b[:, 4] < 0)
D_b[:, 5] = (D_b[:, 5] < 0)
D_m = D_b.astype('str')
D_m[:, 1] = np.array(list(string.ascii_lowercase))[D_b[:, 1].astype('int')]
D_m[:, 4] = np.array(["true", "false"])[D_b[:, 4].astype('int')]
D_m[:, 5] = np.array(["negative", "positive"])[D_b[:, 5].astype('int')]
date_index_d = np.random.randint(0, 366, (num_total, 1))
date_range = np.arange('2010-08-16', '2011-08-17', dtype='datetime64[D]')
d_date = date_range[date_index_d]
data = np.c_[d_date.astype('str'), 
          d_name, 
          d_gender, 
          d_age.astype('str'), 
          D_m]

unknowns_d = np.random.choice(np.arange(num_total),
                              size=8,
                              replace=False)
data[unknowns_d, 9] = "_"

data1 = data[:42, :]

data2 = data[42:, np.array([0, 1, 2, 3, 5, 6, 4, 7, 9])]
data2[:, 6] = (100 * data2[:, 6].astype('float')).astype('str')

missings_d25 = np.random.choice(np.where(data2[:, 3].astype('float') > 40)[0], 
                                size=10, 
                                replace=False)
data2[missings_d25, 5] = "_"

missings_d24 = np.random.choice(np.arange(data2.shape[0]),
                                size=5,
                                replace=False)
data2[missings_d24, 4] = "_"

repeateds_d2 = np.random.choice(np.arange(data2.shape[0]),
                                size=3,
                                replace=False)
data2 = np.r_[data2, data2[repeateds_d2, :]]
data2 = data2[np.random.permutation(data2.shape[0])]


feature_names1 = ["Examination Date", "Name", "Gender", "Age", "M1", "Mode", "Q", "DD", "C"]
label_name1 = "Diagnosis"
column_names1 = feature_names1 + [label_name1]
df_c1 = pd.DataFrame(data1, columns=column_names1)
df_c1 = df_c1.sort_values("Examination Date")
df_c1.index = np.arange(len(df_c1))

feature_names2 = ["Examination Date", "Name", "Gender", "Age", "Mode", "Q", "M1", "DD"]
label_name2 = "Diagnosis"
column_names2 = feature_names2 + [label_name2]
df_c2 = pd.DataFrame(data2, columns=column_names2)

df_c2.loc[[3, 44], "Gender"] = "Male"
df_c2.loc[56, "Gender"] = "fmeale"
df_c2.loc[31, "Age"] = 592
df_c2.loc[[31, 32, 33], "Mode"] = "F"

num_maintenance = 22

date_index_m = np.random.randint(0, 366, (num_maintenance, 1))
m_date = date_range[date_index_m]
M_r = np.random.random((num_maintenance, 4))
M_r[:, 0] *= 9.88e-1
M_r[:, 1] *= 1.05e-2
M_r[:, 2] *= 1.21e+1
M_r[:, 3] *= 2.80e-4
clinic_names = np.array(["clinic1", "clinic2"])
clinic_indices = np.random.randint(0, 2, (num_maintenance, 1))
M_s = clinic_names[clinic_indices]
M = np.c_[m_date.astype('str'), M_r.astype('str'), M_s]
missings_m = np.random.choice(np.arange(1, num_maintenance - 1),
                              size=7,
                              replace=False)
M[missings_m, 1] = "_"

column_names3 = ["Inspection Date", "R1", "R2", "R3", "R4", "Device Site"]
df_m = pd.DataFrame(M, columns=column_names3)

df_m = df_m.sort_values("Inspection Date")
df_m.index = np.arange(len(df_m))

In this example we have three datasets, two datasets from two different hypothetical clincs "clinic1" and "clinic2" which diagnose patients with a novel device which takes a number of measurements . The final goal is to see if they have a certain disease or not. Measurements taken from patients in the two clincs are presented in dataframes `df_c1` and `df_c2`. We also have an inspection log, recorded in `df_m`, for the devices used in "clinic1" and "clinic2" where a number of variables from the the device are measured. Two of these variables, **R1** and **R3**, are believed to affect the readings taken from the patients (the other tow readings are not relevant). The `df_c1` and `df_c2` datasets are labeled with an actual diagnosis whether the patient had the disease or not and the goal is to predict the existence of disease based on the measurements taken from the patients. Since the variables of the devices, measured in inspection, affects the measurements taken from patients in clincs, they should also be considered. Here are the data frames:

In [3]:
display(df_c1)

,Examination Date,Name,Gender,Age,M1,Mode,Q,DD,C,Diagnosis
0,2010-08-16,Charles Cappello,female,56,0.05316955012615155,c,102.653987096958,-0.17703413296000092,false,negative
1,2010-08-26,Florence Fulmer,female,6,0.04302904912234308,e,111.5181976095155,5.969760481174799,true,negative
2,2010-08-29,Marcus Thompson,male,9,0.055075469899240496,f,135.89602624023283,0.4136345340491221,false,positive
3,2010-08-31,Joseph Kasten,male,14,0.0644801181839231,f,128.5880204275917,-1.3426952475666951,false,positive
4,2010-09-26,Glenda Obrien,female,77,0.055257419136733196,f,95.35382812216915,1.5366386265272902,false,positive
5,2010-09-27,Lance Bowen,male,25,0.06941858466013445,e,84.23277651927087,-0.07399496132124983,false,negative
6,2010-11-02,Miriam Madison,female,49,0.04815108144279935,g,95.077136868126,5.715075155513354,true,negative
7,2010-11-08,Lionel Mitchell,male,79,0.06225267204442809,e,125.81484903678188,0.985283341392392,true,negative
8,2010-11-10,Elvia Blaser,female,33,0.047089009871505445,f,87.85031905968846,3.4800988029958324,false,negative
9,2010-11-20,Brandon Stagner,male,8,0.06912600839428865,f,115.82125640841578,0.49819011421869686,true,negative


In [4]:
display(df_c2)

,Examination Date,Name,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,Shane Hemmer,male,72,d,_,5.0589929474594815,1.4818774966932846,negative
1,2011-02-20,Zachary Morgan,male,9,e,104.08763319576146,6.5317244089147515,2.266883908403935,negative
2,2011-06-27,Dora Pruneda,female,59,f,102.90431521853222,6.273313141287465,0.3963334360324253,positive
3,2010-09-22,Myron Mcfarland,Male,23,g,152.651655330603,7.333625993664482,0.557534086789746,positive
4,2011-06-10,Shane Hemmer,male,72,d,_,5.0589929474594815,1.4818774966932846,negative
...,...,...,...,...,...,...,...,...,...
56,2011-08-04,Magdalena Tanner,fmeale,42,e,109.86630676676552,4.137635695184735,6.405663142307764,negative
57,2011-08-14,Dennis Sydow,male,37,e,100.3791042016687,6.295157824980686,-2.297058868668793,positive
58,2011-04-19,Tony Dunson,male,77,f,_,5.52466092336174,3.867671528998077,negative
59,2011-05-29,Jose Modlin,male,35,e,98.54798923423405,2.6886698625786356,1.8975949475179326,positive


In [5]:
display(df_m)

,Inspection Date,R1,R2,R3,R4,Device Site
0,2010-09-04,0.07960088091996784,0.007343563647436414,0.542920687919599,7.42982526466514e-05,clinic1
1,2010-09-27,_,0.00020410051911929706,9.613118162559806,4.231921433232006e-05,clinic1
2,2010-10-06,_,0.0009278592336655931,4.298943320088499,4.135263646543309e-05,clinic1
3,2010-10-21,0.8776172211437634,0.002254392251761286,9.240019315339634,0.0001816752386121514,clinic1
4,2010-11-26,_,0.008547135849330464,6.659527875519007,2.2120270388637947e-05,clinic1
5,2010-11-29,0.7518246219719119,0.006172265431139128,7.023102711734743,1.59905553064144e-05,clinic1
6,2011-01-04,0.9279763002997659,0.0008328473394231052,5.473453924766556,0.0001238036083787162,clinic2
7,2011-01-28,0.9051137625634268,0.008792833619916184,7.223218151610316,0.0002089661552856568,clinic2
8,2011-02-03,0.09188005264255929,0.0015294911720306957,2.745545094647683,2.3601512840388506e-05,clinic1
9,2011-02-04,_,0.007506517738028728,4.4720938449538465,3.5421582244979815e-05,clinic2


`df_c1` is clean, however, `df_c2` and `df_m` need cleaning and handling missing values (indicated by "_") before we get to align the datasets and create a single consolidated dataset. 

## Cleaning `df_c2`

In [6]:
pd.set_option('display.max_rows', 100)

In [7]:
display(df_c2)

,Examination Date,Name,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,Shane Hemmer,male,72,d,_,5.0589929474594815,1.4818774966932846,negative
1,2011-02-20,Zachary Morgan,male,9,e,104.08763319576146,6.5317244089147515,2.266883908403935,negative
2,2011-06-27,Dora Pruneda,female,59,f,102.90431521853222,6.273313141287465,0.3963334360324253,positive
3,2010-09-22,Myron Mcfarland,Male,23,g,152.651655330603,7.333625993664482,0.557534086789746,positive
4,2011-06-10,Shane Hemmer,male,72,d,_,5.0589929474594815,1.4818774966932846,negative
5,2011-07-21,Debra Jackson,female,17,g,67.23905430892725,3.642516477171948,3.76570561777591,positive
6,2011-05-06,Irene Lucas,female,19,_,54.05030860320373,7.1933179647326035,-0.17391497346360185,_
7,2011-07-05,Michael West,male,5,f,47.046445574350194,4.808862934600719,6.446874146830622,negative
8,2010-10-31,Jolene Morrow,female,41,f,_,7.637614040970911,-0.655883875848791,negative
9,2011-05-01,Andrea Crouse,female,36,f,109.64572405866217,7.006120257736223,2.4050819578055194,negative


In [8]:
feature_names_c2 = ["Examination Date", "Name", "Gender", "Age", "Mode", "Q", \
                    "M1", "DD"]
label_name_c2 = "Diagnosis"
column_names_c2 = feature_names_c2 + [label_name_c2]

In [9]:
n_c2 = len(df_c2)
m_c2 = len(df_c2.columns) - 1

print("Number of Examples:", n_c2)
print("Number Features:", m_c2)

Number of Examples: 61
Number Features: 8


In [10]:
fig = px.scatter_matrix(df_c2, dimensions=column_names_c2, color=label_name_c2)

fig.update_layout(width=(m_c2 + 1) * 200,
                 height=(m_c2 + 1) * 200,
                 margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [11]:
numeric_features_c2 = ["Age", "Q", "M1", "DD"]
non_numeric_features_c2 = ["Examination Date", "Name", "Gender", "Mode"]
for feature in feature_names_c2:
  if feature in numeric_features_c2:
    non_numerical_rows = pd.to_numeric(df_c2[feature], errors='coerce').isnull()
    df_non_numeric = df_c2[non_numerical_rows]
    df_numeric = df_c2[~non_numerical_rows]
    fig = px.histogram(df_numeric, 
                      x=feature,
                      color=label_name_c2,
                      marginal="box")
    fig.update_layout(height=400,
                     margin=dict(l=0, r=0, t=100, b=0), 
                     title=feature + ": numeric")
    fig.show()
  else:
    df_non_numeric = df_c2
  fig = px.histogram(df_non_numeric, x=feature, color=label_name_c2)
  fig.update_layout(height=400,
                   margin=dict(l=0, r=0, t=100, b=0), 
                   title=feature + ": non-numeric")

  fig.show()

In [12]:
fig = px.histogram(df_c2, x=label_name_c2)
fig.show()

In [13]:
df_c2_clean = df_c2.copy()

In [14]:
df_c2_clean.drop("Name", axis=1, inplace=True)

In [15]:
missing_label_rows = df_c2_clean[df_c2_clean[label_name_c2] == "_"].index
df_c2_clean.drop(missing_label_rows, axis=0, inplace=True)

In [16]:
df_c2_clean["Gender"].replace("fmeale", "female", inplace=True)
df_c2_clean["Gender"].replace("Male", "male", inplace=True)

df_c2_clean["Mode"].replace("F", "f", inplace=True)

In [17]:
## This will cause an error
# age_outlier_rows = df_c2_clean[df_c2_clean["Age"] > 150]

In [18]:
df_c2_clean.dtypes

Examination Date    object
Gender              object
Age                 object
Mode                object
Q                   object
M1                  object
DD                  object
Diagnosis           object
dtype: object

In [19]:
df_c2_clean["Examination Date"] = \
  pd.to_datetime(df_c2_clean["Examination Date"])

In [20]:
gender_type = pd.CategoricalDtype(categories=["female", "male"])

df_c2_clean["Gender"] = df_c2_clean["Gender"].astype(gender_type)

In [21]:
df_c2_clean["Age"] = df_c2_clean["Age"].astype("int64")

In [22]:
mode_categories = ["a", "b", "c", "d", "e", "f", "g", "h", "_"]
mode_type = pd.CategoricalDtype(categories=mode_categories)

df_c2_clean["Mode"] = df_c2_clean["Mode"].astype(mode_type)

In [23]:
df_c2_clean["Q"] = pd.to_numeric(df_c2_clean["Q"], errors='coerce')

In [24]:
df_c2_clean["M1"] = pd.to_numeric(df_c2_clean["M1"])

In [25]:
df_c2_clean["DD"] = pd.to_numeric(df_c2_clean["DD"])

In [26]:
diagnosis_type = pd.CategoricalDtype(categories=["negative", "positive"])

df_c2_clean["Diagnosis"] = df_c2_clean["Diagnosis"].astype(diagnosis_type)

In [27]:
df_c2_clean.dtypes

Examination Date    datetime64[ns]
Gender                    category
Age                          int64
Mode                      category
Q                          float64
M1                         float64
DD                         float64
Diagnosis                 category
dtype: object

In [28]:
age_outlier_rows = df_c2_clean[df_c2_clean["Age"] > 150].index
df_c2_clean = df_c2_clean.drop(age_outlier_rows, axis=0)

In [29]:
display(df_c2_clean)

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive


In [30]:
duplicate_rows = df_c2_clean.duplicated()

display(duplicate_rows)

0     False
1     False
2     False
3     False
4      True
5     False
7     False
8     False
9     False
11    False
12    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
25    False
26    False
27    False
28    False
29    False
30    False
32     True
33    False
34    False
35    False
36     True
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
60    False
dtype: bool

In [31]:
df_c2_clean = df_c2_clean[~duplicate_rows]

In [32]:
display(df_c2_clean)

,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


In [33]:
df_c2_clean.reset_index(inplace=True)

In [34]:
display(df_c2_clean)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


## Handling missing values of `df_c2`

### Listwise deletion

In [35]:
q_missing = df_c2_clean["Q"].isnull() #You can also use .isna()

display(q_missing)

0      True
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24     True
25     True
26     True
27    False
28    False
29    False
30    False
31    False
32    False
33     True
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45     True
46    False
47    False
48    False
49    False
50     True
51    False
52    False
Name: Q, dtype: bool

In [36]:
df_temp = df_c2_clean[~q_missing].copy()

df_temp.reset_index(inplace=True)

display(df_temp)

,level_0,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
1,2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
2,3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
3,4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
4,5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
5,7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
6,8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
7,9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative
8,10,14,2011-06-16,male,4,f,146.325322,4.370916,-5.119345,positive
9,11,15,2011-06-19,female,32,e,63.210088,5.253893,5.946163,negative


### Dropping the feature (variable)

In [37]:
df_temp = df_c2_clean.drop("Q", axis=1)

display(df_temp)

,index,Examination Date,Gender,Age,Mode,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,7.817222,5.456586,negative


### Imputing with constant

#### Zero

In [38]:
df_temp = df_c2_clean.copy()
df_temp.loc[q_missing, "Q"] = 0

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,0.000000,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,0.000000,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### Mean

In [39]:
df_temp = df_c2_clean.copy()
df_temp.loc[q_missing, "Q"] = df_temp["Q"].mean()

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,104.269683,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,104.269683,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### Median

In [40]:
df_temp = df_c2_clean.copy()
df_temp.loc[q_missing, "Q"] = df_temp["Q"].median()

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,108.024631,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,108.024631,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### Mode

In [41]:
df_temp = df_c2_clean.copy()
df_temp.loc[q_missing, "Q"] = df_temp["Q"].mode(dropna=True)[0]

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,33.128323,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,33.128323,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### Constant category

In [42]:
mode_missing = (df_c2_clean["Mode"] == "_")

df_temp = df_c2_clean.copy()
df_temp.loc[mode_missing, "Mode"] = "e"

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### Maximum frequency category



In [43]:
df_temp = df_c2_clean.copy()
df_temp.loc[mode_missing, "Mode"] = df_c2_clean["Mode"].mode(dropna=True)[0]

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


#### 'Missing' category

In [44]:
df_temp = df_c2_clean.copy()

df_temp["Mode"] = df_temp["Mode"].astype("str")
df_temp["Mode"].replace("_", "missing", inplace=True)

mode_categories_m = ["a", "b", "c", "d", "e", "f", "g", "h", "missing"]
mode_type_m = pd.CategoricalDtype(categories=mode_categories_m)

df_temp["Mode"] = df_temp["Mode"].astype(mode_type_m)

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis
0,0,2011-06-10,male,72,d,NaN,5.058993,1.481877,negative
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative
6,8,2010-10-31,female,41,f,NaN,7.637614,-0.655884,negative
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative


### Non-constant imputation

Educated guessing and ML algorithms we are not going to cover now.


#### Depending on another feature

In [45]:
df_temp = df_c2_clean.copy()
df_temp["Q Missing"] = df_temp["Q"].isnull()
for feature in ["Gender", "Age", "Mode", "M1", "DD"]:
  fig = px.histogram(df_temp, x=feature, color="Q Missing")
  fig.update_layout(height=300, margin=dict(l=0, r=0, t=0, b=0))
  fig.show()

In [46]:
age_greater_than_40 = (df_temp["Age"] > 40)
mean_q_when_age_gt_40 = df_temp.loc[age_greater_than_40, "Q"].mean()
df_temp.loc[q_missing, "Q"] = mean_q_when_age_gt_40

display(df_temp)

,index,Examination Date,Gender,Age,Mode,Q,M1,DD,Diagnosis,Q Missing
0,0,2011-06-10,male,72,d,107.226907,5.058993,1.481877,negative,True
1,1,2011-02-20,male,9,e,104.087633,6.531724,2.266884,negative,False
2,2,2011-06-27,female,59,f,102.904315,6.273313,0.396333,positive,False
3,3,2010-09-22,male,23,g,152.651655,7.333626,0.557534,positive,False
4,5,2011-07-21,female,17,g,67.239054,3.642516,3.765706,positive,False
5,7,2011-07-05,male,5,f,47.046446,4.808863,6.446874,negative,False
6,8,2010-10-31,female,41,f,107.226907,7.637614,-0.655884,negative,True
7,9,2011-05-01,female,36,f,109.645724,7.006120,2.405082,negative,False
8,11,2011-01-24,male,36,e,106.403539,4.329964,-2.105617,positive,False
9,12,2011-02-04,male,2,f,116.181745,7.817222,5.456586,negative,False


### Adding 'missingness' indicator feature

We add a 'missingness' indicator feature for **Q** which has missing values.

In [47]:
df_temp = df_c2_clean.copy()

df_temp["Mode"] = df_temp["Mode"].astype("str")
df_temp["Mode"].replace("_", "missing", inplace=True)

df_temp["Mode"] = df_temp["Mode"].astype(mode_type_m)

is_q_missing = df_c2_clean["Q"].isnull()
df_temp["Q Missing"] = is_q_missing

df_temp = df_temp[["Examination Date", "Gender", "Age", "Mode", \
                   "Q", "Q Missing", "M1", "DD", "Diagnosis"]]

display(df_temp)

,Examination Date,Gender,Age,Mode,Q,Q Missing,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,True,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,False,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,False,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,False,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,False,3.642516,3.765706,positive
5,2011-07-05,male,5,f,47.046446,False,4.808863,6.446874,negative
6,2010-10-31,female,41,f,NaN,True,7.637614,-0.655884,negative
7,2011-05-01,female,36,f,109.645724,False,7.006120,2.405082,negative
8,2011-01-24,male,36,e,106.403539,False,4.329964,-2.105617,positive
9,2011-02-04,male,2,f,116.181745,False,7.817222,5.456586,negative


In [48]:
df_c2_clean = df_temp.copy()

display(df_c2_clean)

,Examination Date,Gender,Age,Mode,Q,Q Missing,M1,DD,Diagnosis
0,2011-06-10,male,72,d,NaN,True,5.058993,1.481877,negative
1,2011-02-20,male,9,e,104.087633,False,6.531724,2.266884,negative
2,2011-06-27,female,59,f,102.904315,False,6.273313,0.396333,positive
3,2010-09-22,male,23,g,152.651655,False,7.333626,0.557534,positive
4,2011-07-21,female,17,g,67.239054,False,3.642516,3.765706,positive
5,2011-07-05,male,5,f,47.046446,False,4.808863,6.446874,negative
6,2010-10-31,female,41,f,NaN,True,7.637614,-0.655884,negative
7,2011-05-01,female,36,f,109.645724,False,7.006120,2.405082,negative
8,2011-01-24,male,36,e,106.403539,False,4.329964,-2.105617,positive
9,2011-02-04,male,2,f,116.181745,False,7.817222,5.456586,negative


## Cleaning `df_m`

In [49]:
display(df_m)

,Inspection Date,R1,R2,R3,R4,Device Site
0,2010-09-04,0.07960088091996784,0.007343563647436414,0.542920687919599,7.42982526466514e-05,clinic1
1,2010-09-27,_,0.00020410051911929706,9.613118162559806,4.231921433232006e-05,clinic1
2,2010-10-06,_,0.0009278592336655931,4.298943320088499,4.135263646543309e-05,clinic1
3,2010-10-21,0.8776172211437634,0.002254392251761286,9.240019315339634,0.0001816752386121514,clinic1
4,2010-11-26,_,0.008547135849330464,6.659527875519007,2.2120270388637947e-05,clinic1
5,2010-11-29,0.7518246219719119,0.006172265431139128,7.023102711734743,1.59905553064144e-05,clinic1
6,2011-01-04,0.9279763002997659,0.0008328473394231052,5.473453924766556,0.0001238036083787162,clinic2
7,2011-01-28,0.9051137625634268,0.008792833619916184,7.223218151610316,0.0002089661552856568,clinic2
8,2011-02-03,0.09188005264255929,0.0015294911720306957,2.745545094647683,2.3601512840388506e-05,clinic1
9,2011-02-04,_,0.007506517738028728,4.4720938449538465,3.5421582244979815e-05,clinic2


In [50]:
feature_names_m = ["Inspection Date", "R1", "R2", "R3", "R4", "Device Site"]
column_names_m = feature_names_m

In [51]:
n_m = len(df_m)
m_m = len(df_m.columns)

print("Number of Examples:", n_m)
print("Number Features:", m_m)

Number of Examples: 22
Number Features: 6


In [52]:
fig = px.scatter_matrix(df_m, dimensions=column_names_m)

fig.update_layout(width=(m_m + 1) * 200,
                 height=(m_m + 1) * 200,
                 margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [53]:
numeric_features_m = ["R1", "R2", "R3", "R4"]
non_numeric_features_m = ["Inspection Date", "Device Site"]
for feature in feature_names_m:
  if feature in numeric_features_m:
    non_numerical_rows = pd.to_numeric(df_m[feature], errors='coerce').isnull()
    df_non_numeric = df_m[non_numerical_rows]
    df_numeric = df_m[~non_numerical_rows]
    fig = px.histogram(df_numeric, 
                      x=feature,
                      marginal="box")
    fig.update_layout(height=400,
                     margin=dict(l=0, r=0, t=100, b=0), 
                     title=feature + ": numeric")
    fig.show()
  else:
    df_non_numeric = df_m
  fig = px.histogram(df_non_numeric, x=feature)
  fig.update_layout(height=400,
                   margin=dict(l=0, r=0, t=100, b=0), 
                   title=feature + ": non-numeric")

  fig.show()

In [54]:
df_m_clean = df_m.copy()

In [55]:
df_m_clean.drop(["R2", "R4"], axis=1, inplace=True)

In [56]:
df_m_clean.dtypes

Inspection Date    object
R1                 object
R3                 object
Device Site        object
dtype: object

In [57]:
df_m_clean["Inspection Date"] = \
  pd.to_datetime(df_m_clean["Inspection Date"])

In [58]:
df_m_clean["R1"] = pd.to_numeric(df_m_clean["R1"], errors='coerce')

In [59]:
df_m_clean["R3"] = pd.to_numeric(df_m_clean["R3"])

In [60]:
site_type = pd.CategoricalDtype(categories=["clinic1", "clinic2"])

df_m_clean["Device Site"] = df_m_clean["Device Site"].astype(site_type)

In [61]:
df_m_clean.dtypes

Inspection Date    datetime64[ns]
R1                        float64
R3                        float64
Device Site              category
dtype: object

In [62]:
duplicate_rows = df_m_clean.duplicated()

display(duplicate_rows.any())

False

In [63]:
display(df_m_clean)

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,NaN,9.613118,clinic1
2,2010-10-06,NaN,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,NaN,6.659528,clinic1
5,2010-11-29,0.751825,7.023103,clinic1
6,2011-01-04,0.927976,5.473454,clinic2
7,2011-01-28,0.905114,7.223218,clinic2
8,2011-02-03,0.091880,2.745545,clinic1
9,2011-02-04,NaN,4.472094,clinic2


In [64]:
df_m_clean.drop(9, axis=0, inplace=True)

In [65]:
df_m_clean.reset_index(inplace=True, drop=True)

In [66]:
display(df_m_clean)

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,NaN,9.613118,clinic1
2,2010-10-06,NaN,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,NaN,6.659528,clinic1
5,2010-11-29,0.751825,7.023103,clinic1
6,2011-01-04,0.927976,5.473454,clinic2
7,2011-01-28,0.905114,7.223218,clinic2
8,2011-02-03,0.091880,2.745545,clinic1
9,2011-02-04,0.683924,5.047370,clinic2


## Handling missing values of `df_m`

In [67]:
df_m1_clean = df_m_clean[df_m_clean["Device Site"] == 'clinic1'].copy()
df_m2_clean = df_m_clean[df_m_clean["Device Site"] == 'clinic2'].copy()

df_m1_clean.sort_values(by="Inspection Date", inplace=True)
df_m2_clean.sort_values(by="Inspection Date", inplace=True)

df_m1_clean.reset_index(inplace=True, drop=True)
df_m2_clean.reset_index(inplace=True, drop=True)

In [68]:
display(df_m1_clean)

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,NaN,9.613118,clinic1
2,2010-10-06,NaN,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,NaN,6.659528,clinic1
5,2010-11-29,0.751825,7.023103,clinic1
6,2011-02-03,0.091880,2.745545,clinic1
7,2011-02-17,0.603187,5.686296,clinic1
8,2011-03-10,0.510847,11.652810,clinic1
9,2011-05-21,0.825806,11.933157,clinic1


In [69]:
display(df_m2_clean)

,Inspection Date,R1,R3,Device Site
0,2011-01-04,0.927976,5.473454,clinic2
1,2011-01-28,0.905114,7.223218,clinic2
2,2011-02-04,0.683924,5.047370,clinic2
3,2011-02-27,0.622961,1.971804,clinic2
4,2011-04-10,NaN,10.822079,clinic2
5,2011-05-21,NaN,5.152056,clinic2
6,2011-05-30,0.498139,7.036262,clinic2
7,2011-07-17,0.638587,10.053019,clinic2


### Last Observation Carried Forward (LOCF)

In [70]:
df_temp = df_m1_clean.copy()

df_temp.fillna(method='ffill', inplace=True)

display(df_temp)

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,0.079601,9.613118,clinic1
2,2010-10-06,0.079601,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,0.877617,6.659528,clinic1
5,2010-11-29,0.751825,7.023103,clinic1
6,2011-02-03,0.091880,2.745545,clinic1
7,2011-02-17,0.603187,5.686296,clinic1
8,2011-03-10,0.510847,11.652810,clinic1
9,2011-05-21,0.825806,11.933157,clinic1


### Next Observation Carried Backward (NOCB)



In [71]:
df_temp = df_m2_clean.copy()

df_temp.fillna(method='bfill', inplace=True)

display(df_temp)

,Inspection Date,R1,R3,Device Site
0,2011-01-04,0.927976,5.473454,clinic2
1,2011-01-28,0.905114,7.223218,clinic2
2,2011-02-04,0.683924,5.047370,clinic2
3,2011-02-27,0.622961,1.971804,clinic2
4,2011-04-10,0.498139,10.822079,clinic2
5,2011-05-21,0.498139,5.152056,clinic2
6,2011-05-30,0.498139,7.036262,clinic2
7,2011-07-17,0.638587,10.053019,clinic2


### Linear interpolation



In [72]:
df_temp = df_m1_clean.copy()

df_temp.set_index("Inspection Date", inplace=True)

display(df_temp)

,R1,R3,Device Site
Inspection Date,,,
2010-09-04,0.079601,0.542921,clinic1
2010-09-27,NaN,9.613118,clinic1
2010-10-06,NaN,4.298943,clinic1
2010-10-21,0.877617,9.240019,clinic1
2010-11-26,NaN,6.659528,clinic1
2010-11-29,0.751825,7.023103,clinic1
2011-02-03,0.091880,2.745545,clinic1
2011-02-17,0.603187,5.686296,clinic1
2011-03-10,0.510847,11.652810,clinic1


In [73]:
df_temp["R1"].interpolate(method='index', inplace=True)

df_temp.reset_index(inplace=True)

display(df_temp)

,Inspection Date,R1,R3,Device Site
0,2010-09-04,0.079601,0.542921,clinic1
1,2010-09-27,0.470120,9.613118,clinic1
2,2010-10-06,0.622931,4.298943,clinic1
3,2010-10-21,0.877617,9.240019,clinic1
4,2010-11-26,0.761501,6.659528,clinic1
5,2010-11-29,0.751825,7.023103,clinic1
6,2011-02-03,0.091880,2.745545,clinic1
7,2011-02-17,0.603187,5.686296,clinic1
8,2011-03-10,0.510847,11.652810,clinic1
9,2011-05-21,0.825806,11.933157,clinic1


## Inspecting `df_c1`

In [74]:
display(df_c1)

,Examination Date,Name,Gender,Age,M1,Mode,Q,DD,C,Diagnosis
0,2010-08-16,Charles Cappello,female,56,0.05316955012615155,c,102.653987096958,-0.17703413296000092,false,negative
1,2010-08-26,Florence Fulmer,female,6,0.04302904912234308,e,111.5181976095155,5.969760481174799,true,negative
2,2010-08-29,Marcus Thompson,male,9,0.055075469899240496,f,135.89602624023283,0.4136345340491221,false,positive
3,2010-08-31,Joseph Kasten,male,14,0.0644801181839231,f,128.5880204275917,-1.3426952475666951,false,positive
4,2010-09-26,Glenda Obrien,female,77,0.055257419136733196,f,95.35382812216915,1.5366386265272902,false,positive
5,2010-09-27,Lance Bowen,male,25,0.06941858466013445,e,84.23277651927087,-0.07399496132124983,false,negative
6,2010-11-02,Miriam Madison,female,49,0.04815108144279935,g,95.077136868126,5.715075155513354,true,negative
7,2010-11-08,Lionel Mitchell,male,79,0.06225267204442809,e,125.81484903678188,0.985283341392392,true,negative
8,2010-11-10,Elvia Blaser,female,33,0.047089009871505445,f,87.85031905968846,3.4800988029958324,false,negative
9,2010-11-20,Brandon Stagner,male,8,0.06912600839428865,f,115.82125640841578,0.49819011421869686,true,negative


In [75]:
feature_names_c1 = ["Examination Date", "Name", "Gender", "Age", "Mode", "Q", \
                    "M1", "DD", "C"]
label_name_c1 = "Diagnosis"
column_names_c1 = feature_names_c1 + [label_name_c1]

In [76]:
n_c1 = len(df_c1)
m_c1 = len(df_c1.columns) - 1

print("Number of Examples:", n_c1)
print("Number Features:", m_c1)

Number of Examples: 42
Number Features: 9


In [77]:
fig = px.scatter_matrix(df_c1, dimensions=column_names_c1, color=label_name_c1)

fig.update_layout(width=(m_c1 + 1) * 200,
                 height=(m_c1 + 1) * 200,
                 margin=dict(l=0, r=0, t=0, b=0))

fig.show()

In [78]:
numeric_features_c1 = ["Age", "Q", "M1", "DD"]
non_numeric_features_c1 = ["Examination Date", "Name", "Gender", "Mode", "C"]
for feature in feature_names_c1:
  if feature in numeric_features_c1:
    fig = px.histogram(df_c1, x=feature, color=label_name_c1, marginal="box")
    fig.update_layout(height=300, margin=dict(l=0, r=0, t=100, b=0))
  else:
    fig = px.histogram(df_c1, x=feature, color=label_name_c1)
    fig.update_layout(height=300, margin=dict(l=0, r=0, t=0, b=0))
  fig.show()

In [79]:
fig = px.histogram(df_c1, x=label_name_c1)
fig.show()

In [80]:
df_c1_clean = df_c1.copy()

In [81]:
df_c1_clean.drop("Name", axis=1, inplace=True)

In [82]:
missing_label_rows = df_c1_clean[df_c1_clean[label_name_c1] == "_"].index
df_c1_clean.drop(missing_label_rows, axis=0, inplace=True)

In [83]:
df_c1_clean.dtypes

Examination Date    object
Gender              object
Age                 object
M1                  object
Mode                object
Q                   object
DD                  object
C                   object
Diagnosis           object
dtype: object

In [84]:
df_c1_clean["Examination Date"] = \
  pd.to_datetime(df_c1_clean["Examination Date"])

In [85]:
df_c1_clean["Gender"] = df_c1_clean["Gender"].astype(gender_type)

In [86]:
df_c1_clean["Age"] = df_c1_clean["Age"].astype("int64")

In [87]:
df_c1_clean["Mode"] = df_c1_clean["Mode"].astype(mode_type)

In [88]:
df_c1_clean["Q"] = pd.to_numeric(df_c1_clean["Q"])

In [89]:
df_c1_clean["M1"] = pd.to_numeric(df_c1_clean["M1"])

In [90]:
df_c1_clean["DD"] = pd.to_numeric(df_c1_clean["DD"])

In [91]:
df_c1_clean["C"] = df_c1_clean["C"].astype("bool")

In [92]:
df_c1_clean["Diagnosis"] = df_c1_clean["Diagnosis"].astype(diagnosis_type)

In [93]:
df_c1_clean.dtypes

Examination Date    datetime64[ns]
Gender                    category
Age                          int64
M1                         float64
Mode                      category
Q                          float64
DD                         float64
C                             bool
Diagnosis                 category
dtype: object

In [94]:
display(df_c1_clean)

,Examination Date,Gender,Age,M1,Mode,Q,DD,C,Diagnosis
0,2010-08-16,female,56,0.053170,c,102.653987,-0.177034,True,negative
1,2010-08-26,female,6,0.043029,e,111.518198,5.969760,True,negative
2,2010-08-29,male,9,0.055075,f,135.896026,0.413635,True,positive
3,2010-08-31,male,14,0.064480,f,128.588020,-1.342695,True,positive
4,2010-09-26,female,77,0.055257,f,95.353828,1.536639,True,positive
5,2010-09-27,male,25,0.069419,e,84.232777,-0.073995,True,negative
6,2010-11-02,female,49,0.048151,g,95.077137,5.715075,True,negative
7,2010-11-08,male,79,0.062253,e,125.814849,0.985283,True,negative
8,2010-11-10,female,33,0.047089,f,87.850319,3.480099,True,negative
9,2010-11-20,male,8,0.069126,f,115.821256,0.498190,True,negative


## Aligning datasets

### Preparation

Feature **M1** in `df_c1` is represented in units that are 100 times larger than **M1** in `df_c2` (the values in `df_c1` are 100 times smaller than those of `df_c2`). All other values are in the same units. Let's scale the **M1** values in `df_c1` to match:


In [ ]:
df_c1_clean["M1"] *= 100

display(df_c1_clean)

In [ ]:
df_c1_clean.rename(columns={"Examination Date": "Date"}, inplace=True)
df_c2_clean.rename(columns={"Examination Date": "Date"}, inplace=True)
df_m1_clean.rename(columns={"Inspection Date": "Date"}, inplace=True)
df_m2_clean.rename(columns={"Inspection Date": "Date"}, inplace=True)

In [ ]:
display(df_c1_clean.columns)
display(df_c2_clean.columns)
display(df_m1_clean.columns)
display(df_m2_clean.columns)

### Aligning

In [ ]:
df_cm1 = pd.concat([df_c1_clean, df_m1_clean], axis=0)

display(df_cm1)

In [ ]:
df_cm1.sort_values("Date", inplace=True)

display(df_cm1)

In [ ]:
df_cm1.set_index("Date", inplace=True)
df_cm1["R1"].interpolate(method='index', inplace=True)
df_cm1["R3"].interpolate(method='index', inplace=True)

display(df_cm1)

In [ ]:
df_cm1["R1"].fillna(method='bfill', inplace=True)
df_cm1["R3"].fillna(method='bfill', inplace=True)

display(df_cm1)

In [ ]:
df_cm1 = df_cm1[df_cm1["Device Site"].isnull()]

display(df_cm1)

In [ ]:
df_cm2 = pd.concat([df_c2_clean, df_m2_clean], axis=0)

df_cm2.sort_values("Date", inplace=True)

df_cm2.set_index("Date", inplace=True)

df_cm2["R1"].interpolate(method='index', inplace=True)
df_cm2["R3"].interpolate(method='index', inplace=True)

display(df_cm2)

In [ ]:
df_cm2["R1"].fillna(method='bfill', inplace=True)
df_cm2["R3"].fillna(method='bfill', inplace=True)

df_cm2 = df_cm2[df_cm2["Device Site"].isnull()]

display(df_cm2)

In [ ]:
df_cm1["Q Missing"] = False
df_cm1["C Missing"] = False

df_cm2["C"] = False
df_cm2["C Missing"] = True

In [ ]:
df = pd.concat([df_cm1, df_cm2], axis=0)

display(df)

In [ ]:
df.sort_values("Date", inplace=True)

df = df[["Gender", "Age", "M1", "Mode", "Q", "Q Missing", "DD", "C", \
         "C Missing", "R1", "R3", "Diagnosis"]]

display(df)

In [ ]:
df.reset_index(inplace=True)

display(df)

In [ ]:
df.drop("Date", axis=1, inplace=True)

In [ ]:
q_missing = df["Q"].isnull()
age_greater_than_40 = (df["Age"] > 40)
mean_q_when_age_gt_40 = df.loc[age_greater_than_40, "Q"].mean()
df.loc[q_missing, "Q"] = mean_q_when_age_gt_40

display(df)

## Feature encoding

### Converting to numbers

In [ ]:
df = pd.get_dummies(df, columns=["Mode"], prefix='mode')

In [ ]:
display(df)

In [ ]:
df = pd.get_dummies(df, columns=["Gender"], prefix='gender', drop_first=True)
df = pd.get_dummies(df, columns=["Diagnosis"], prefix='diagnosis', drop_first=True)

display(df)

In [ ]:
df.rename(columns={"Age": "age", 
                   "M1": "m1", 
                   "Q" : "q", 
                   "Q Missing": "q_missing", 
                   "DD" : "dd", 
                   "C" : "c", 
                   "C Missing": "c_missing",
                   "R1": "r1",
                   "R3": "r3"},
          inplace=True)

df = df[["gender_male", "age", "m1", "mode_a", "mode_b", "mode_c", "mode_d", \
         "mode_e", "mode_f", "mode_g", "mode_h", "mode_missing", "q", \
         "q_missing", "dd", "c", "c_missing", "r1", "r3", "diagnosis_positive"]]

df["age"] = df["age"].astype("uint8")
df["q_missing"] = df["q_missing"].astype("uint8")
df["c"] = df["c"].astype("uint8")
df["c_missing"] = df["c_missing"].astype("uint8")

display(df)

In [ ]:
df.dtypes

In [ ]:
display(df.duplicated().any())

### Normalization and standardization

We will talk about the details of normalization in the 3rd course, but for now, just observe what happens when we do normalization:

In [ ]:
import sklearn.preprocessing

In [ ]:
x_m1 = df[['m1']].values
m1_scaler = sklearn.preprocessing.StandardScaler()
x_m1_scaled = m1_scaler.fit_transform(x_m1)
df["m1"] = x_m1_scaled

display(df["m1"])

In [ ]:
display(x_m1_scaled.mean(), x_m1_scaled.std())

In [ ]:
x_q = df[['q']].values
q_scaler = sklearn.preprocessing.StandardScaler()
x_q_scaled = q_scaler.fit_transform(x_q)
df["q"] = x_q_scaled

In [ ]:
x_dd = df[['dd']].values
dd_scaler = sklearn.preprocessing.MinMaxScaler()
x_dd_scaled = dd_scaler.fit_transform(x_dd)

df["dd"] = (2 * x_dd_scaled) - 1

In [ ]:
display(df)

In [ ]:
display(df['dd'].min(), df['dd'].max())

## Feature transformations

In [ ]:
df["age_lt_25"] = (df["age"] < 25).astype("uint8")
df["age_gt_40"] = (df["age"] > 40).astype("uint8")

display(df)

In [ ]:
df["log_m1"] = np.log(np.abs(df["m1"]))

In [ ]:
q_dd_poly3 = sklearn.preprocessing.PolynomialFeatures(degree=3)

q_dd_d3_columns = ["q0dd0", \
                   "q1dd0", "q0dd1", \
                   "q2dd0", "q1dd1", "q0dd2", \
                   "q3dd0", "q2dd1", "q1dd2", "q0dd3"]

df = df.reindex(columns=list(df.columns) + q_dd_d3_columns)    

df[q_dd_d3_columns] = q_dd_poly3.fit_transform(df[["q", "dd"]])

display(df)

In [ ]:
label_name = "diagnosis_positive"
labels = df[label_name]
df.drop(label_name, axis=1, inplace=True)
feature_names = df.columns
df[label_name] = labels

display(df)

## Feature selection

See:
1. https://scikit-learn.org/stable/modules/feature_selection.html
2. https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest


In [ ]:
for feature in feature_names:
  fig = px.histogram(df, x=feature, color=label_name, marginal="box")
  fig.update_layout(height=300, margin=dict(l=0, r=0, t=0, b=0))
  fig.show()

In [ ]:
import sklearn.feature_selection

In [ ]:
func = sklearn.feature_selection.mutual_info_classif
feature_selector = sklearn.feature_selection.SelectKBest(func, k=6)
X = df.loc[:, df.columns != label_name].values
y = df[label_name].values
feature_selector.fit(X, y)
selected_columns = feature_names[feature_selector.get_support()]
display(df[selected_columns])

That's all Folks!